<a href="https://colab.research.google.com/github/kweenkeen/ReachOutandRead/blob/main/Mail_Merge_Monthly_Email.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import files
from datetime import date
from datetime import datetime

# UPLOAD SITE DATA REPORT

site_data_report = files.upload()
site_data_file = next(iter(site_data_report))
df = pd.read_excel(site_data_file, converters={'Site ID': str})
df = df.set_index('Site ID')

Saving MyROR Site Data Report 2022-07-15.xlsx to MyROR Site Data Report 2022-07-15.xlsx


In [2]:
# UPLOAD AFFILIATE CONTACT LIST

affiliate_contact_list = files.upload()
affiliate_filename = next(iter(affiliate_contact_list))
affiliate_contact_df = pd.read_excel(affiliate_filename)

Saving 2022.07.12 Find a Site Affiliate Contact Information.xlsx to 2022.07.12 Find a Site Affiliate Contact Information.xlsx


In [3]:
# Set filter
active_or_approved = ['Active', 'Active - Non-Reporting',
                      'Slated for deactivation', 'COVID-19 Freeze',
                      'Approved - Needs Training', 'Approved - Waitlisted',
                      'In Progress'] 
df = df[df['Status'].isin(active_or_approved)] 

In [5]:
rslt_df = df[['Affiliate Name', 'Status', 
              'Join Date', 'Training %', 'Total Trained', 
              'Most Recent Quality Classification Date', 
              'Primary Contact Email', 'Medical Consultant Email']]

In [6]:
mail_merge_df = pd.DataFrame(columns=['Affiliate Name'])
mail_merge_df = mail_merge_df.set_index('Affiliate Name')
mail_merge_df.head()

""
Affiliate Name


In [7]:
high_level_active = ['Active', 'Active - Non-Reporting', 
                     'Slated for deactivation', 'COVID-19 Freeze'] 
active_df = rslt_df[rslt_df['Status'].isin(high_level_active)] 
active_sites_df = active_df['Affiliate Name'].value_counts()
active_sites_df =active_sites_df.transpose()

In [8]:
active_sites_df.head()

Independent / International          656
Reach Out and Read North Carolina    375
Reach Out and Read Colorado          329
Reach Out and Read Minnesota         301
Reach Out and Read Massachusetts     268
Name: Affiliate Name, dtype: int64

In [9]:
mail_merge_df['Number of Sites (High Level Active)'] = active_sites_df

In [10]:
mail_merge_df.head()

,Number of Sites (High Level Active)
Affiliate Name,
Independent / International,656
Reach Out and Read North Carolina,375
Reach Out and Read Colorado,329
Reach Out and Read Minnesota,301
Reach Out and Read Massachusetts,268


In [11]:
date_range_min = input("Type in the start of your year (MM/DD/YYYY): ")
date_range_min = datetime.strptime(date_range_min, '%m/%d/%Y')
date_range_max = input("Type in the end of your year (MM/DD/YYYY): ")
date_range_max = datetime.strptime(date_range_max, '%m/%d/%Y')

Type in the start of your year (MM/DD/YYYY): 07/01/2022
Type in the end of your year (MM/DD/YYYY): 07/15/2022


In [12]:
join_date_df = rslt_df.loc[(rslt_df['Join Date'] >= date_range_min) & (rslt_df['Join Date'] <= date_range_max)]
join_date_df['Join Date'] = join_date_df['Join Date'].dt.date

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
new_sites_df = join_date_df['Affiliate Name'].value_counts()
mail_merge_df['Number of New Sites'] = new_sites_df
mail_merge_df['Number of New Sites'] = mail_merge_df['Number of New Sites'].fillna(0)
mail_merge_df['Number of New Sites'] = mail_merge_df['Number of New Sites'].astype(int)
mail_merge_df.head()

,Number of Sites (High Level Active),Number of New Sites
Affiliate Name,,
Independent / International,656,1
Reach Out and Read North Carolina,375,0
Reach Out and Read Colorado,329,1
Reach Out and Read Minnesota,301,1
Reach Out and Read Massachusetts,268,2


In [14]:
training_percent_average_df = active_df.groupby('Affiliate Name')['Training %'].mean()

In [15]:
training_percent_average_df.head()

Affiliate Name
Delaware (TIROS 1)                     0.220000
Idaho (Curiosity Rover)                0.760909
Independent / International            0.686371
Michigan (Mars Pathfinder)             0.788969
PA (excluding Greater Philadelphia)    0.826892
Name: Training %, dtype: float64

In [16]:
mail_merge_df['Current Overall Training Percentage'] = training_percent_average_df
mail_merge_df['Current Overall Training Percentage'] = mail_merge_df['Current Overall Training Percentage']* 100
mail_merge_df['Current Overall Training Percentage'] = mail_merge_df['Current Overall Training Percentage'].astype(int)
mail_merge_df['Current Overall Training Percentage'] = mail_merge_df['Current Overall Training Percentage'].astype(str) + '%'

In [17]:
mail_merge_df.head()

,Number of Sites (High Level Active),Number of New Sites,Current Overall Training Percentage
Affiliate Name,,,
Independent / International,656,1,68%
Reach Out and Read North Carolina,375,0,93%
Reach Out and Read Colorado,329,1,92%
Reach Out and Read Minnesota,301,1,86%
Reach Out and Read Massachusetts,268,2,98%


In [18]:
providers_trained_count_df = active_df.groupby('Affiliate Name')['Total Trained'].sum().astype(int)

In [19]:
providers_trained_count_df.head()

Affiliate Name
Delaware (TIROS 1)                       17
Idaho (Curiosity Rover)                  95
Independent / International            2333
Michigan (Mars Pathfinder)              339
PA (excluding Greater Philadelphia)    1164
Name: Total Trained, dtype: int64

In [20]:
mail_merge_df['Number of Providers Trained'] = providers_trained_count_df

In [21]:
mail_merge_df.head()

,Number of Sites (High Level Active),Number of New Sites,Current Overall Training Percentage,Number of Providers Trained
Affiliate Name,,,,
Independent / International,656,1,68%,2333
Reach Out and Read North Carolina,375,0,93%,2001
Reach Out and Read Colorado,329,1,92%,1635
Reach Out and Read Minnesota,301,1,86%,2385
Reach Out and Read Massachusetts,268,2,98%,1823


In [22]:
SQC_date_df = rslt_df.loc[(rslt_df['Most Recent Quality Classification Date'] >= date_range_min) & (rslt_df['Most Recent Quality Classification Date'] <= date_range_max)]
SQC_date_df['Most Recent Quality Classification Date'] = SQC_date_df['Most Recent Quality Classification Date'].dt.date

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [23]:
SQC_updates_df = SQC_date_df['Affiliate Name'].value_counts()

In [24]:
SQC_updates_df.head()

Reach Out and Read Texas                         11
Reach Out and Read San Diego (California)         7
Reach Out and Read Georgia                        7
Reach Out and Read Orange County (California)     7
Reach Out and Read Washington                     6
Name: Affiliate Name, dtype: int64

In [25]:
mail_merge_df['# of SQCs Completed'] =SQC_updates_df
mail_merge_df['# of SQCs Completed'] = mail_merge_df['# of SQCs Completed'].fillna(0)
mail_merge_df['# of SQCs Completed'] = mail_merge_df['# of SQCs Completed'].astype(int)

In [26]:
mail_merge_df.head()

,Number of Sites (High Level Active),Number of New Sites,Current Overall Training Percentage,Number of Providers Trained,# of SQCs Completed
Affiliate Name,,,,,
Independent / International,656,1,68%,2333,0
Reach Out and Read North Carolina,375,0,93%,2001,0
Reach Out and Read Colorado,329,1,92%,1635,0
Reach Out and Read Minnesota,301,1,86%,2385,0
Reach Out and Read Massachusetts,268,2,98%,1823,5


In [27]:
in_progress_df = df[df['Status'] == 'In Progress'] 
in_progress_df = in_progress_df['Affiliate Name'].value_counts()
in_progress_df = in_progress_df.transpose()
mail_merge_df['# of Applications in Progress'] = in_progress_df
mail_merge_df['# of Applications in Progress'] = mail_merge_df['# of Applications in Progress'].fillna(0)
mail_merge_df['# of Applications in Progress'] = mail_merge_df['# of Applications in Progress'].astype(int)

In [28]:
mail_merge_df.head()

,Number of Sites (High Level Active),Number of New Sites,Current Overall Training Percentage,Number of Providers Trained,# of SQCs Completed,# of Applications in Progress
Affiliate Name,,,,,,
Independent / International,656,1,68%,2333,0,95
Reach Out and Read North Carolina,375,0,93%,2001,0,40
Reach Out and Read Colorado,329,1,92%,1635,0,13
Reach Out and Read Minnesota,301,1,86%,2385,0,17
Reach Out and Read Massachusetts,268,2,98%,1823,5,17


In [29]:
approved_needs_training_df = df[df['Status'] == 'Approved - Needs Training'] 
approved_needs_training_df = approved_needs_training_df['Affiliate Name'].value_counts()
approved_needs_training_df = approved_needs_training_df.transpose()
mail_merge_df['# of Approved - Needs Training Sites'] = approved_needs_training_df
mail_merge_df['# of Approved - Needs Training Sites'] = mail_merge_df['# of Approved - Needs Training Sites'].fillna(0)
mail_merge_df['# of Approved - Needs Training Sites'] = mail_merge_df['# of Approved - Needs Training Sites'].astype(int)

In [30]:
mail_merge_df.head()

,Number of Sites (High Level Active),Number of New Sites,Current Overall Training Percentage,Number of Providers Trained,# of SQCs Completed,# of Applications in Progress,# of Approved - Needs Training Sites
Affiliate Name,,,,,,,
Independent / International,656,1,68%,2333,0,95,56
Reach Out and Read North Carolina,375,0,93%,2001,0,40,13
Reach Out and Read Colorado,329,1,92%,1635,0,13,4
Reach Out and Read Minnesota,301,1,86%,2385,0,17,6
Reach Out and Read Massachusetts,268,2,98%,1823,5,17,3


In [31]:
approved_waitlisted_df = df[df['Status'] == 'Approved - Waitlisted'] 
approved_waitlisted_df = approved_waitlisted_df['Affiliate Name'].value_counts()
approved_waitlisted_df = approved_waitlisted_df.transpose()
mail_merge_df['# of Approved - Waitlisted Sites'] = approved_waitlisted_df
mail_merge_df['# of Approved - Waitlisted Sites'] = mail_merge_df['# of Approved - Waitlisted Sites'].fillna(0)
mail_merge_df['# of Approved - Waitlisted Sites'] = mail_merge_df['# of Approved - Waitlisted Sites'].astype(int)

In [32]:
mail_merge_df.head()

,Number of Sites (High Level Active),Number of New Sites,Current Overall Training Percentage,Number of Providers Trained,# of SQCs Completed,# of Applications in Progress,# of Approved - Needs Training Sites,# of Approved - Waitlisted Sites
Affiliate Name,,,,,,,,
Independent / International,656,1,68%,2333,0,95,56,9
Reach Out and Read North Carolina,375,0,93%,2001,0,40,13,3
Reach Out and Read Colorado,329,1,92%,1635,0,13,4,10
Reach Out and Read Minnesota,301,1,86%,2385,0,17,6,0
Reach Out and Read Massachusetts,268,2,98%,1823,5,17,3,3


In [33]:
covid_freeze_df = df[df['Status'] == 'COVID-19 Freeze'] 
covid_freeze_df = covid_freeze_df['Affiliate Name'].value_counts()
covid_freeze_df = covid_freeze_df.transpose()
mail_merge_df['# of Covid Freeze Sites'] = covid_freeze_df
mail_merge_df['# of Covid Freeze Sites'] = mail_merge_df['# of Covid Freeze Sites'].fillna(0)
mail_merge_df['# of Covid Freeze Sites'] = mail_merge_df['# of Covid Freeze Sites'].astype(int)

In [34]:
mail_merge_df.head()

,Number of Sites (High Level Active),Number of New Sites,Current Overall Training Percentage,Number of Providers Trained,# of SQCs Completed,# of Applications in Progress,# of Approved - Needs Training Sites,# of Approved - Waitlisted Sites,# of Covid Freeze Sites
Affiliate Name,,,,,,,,,
Independent / International,656,1,68%,2333,0,95,56,9,4
Reach Out and Read North Carolina,375,0,93%,2001,0,40,13,3,2
Reach Out and Read Colorado,329,1,92%,1635,0,13,4,10,0
Reach Out and Read Minnesota,301,1,86%,2385,0,17,6,0,1
Reach Out and Read Massachusetts,268,2,98%,1823,5,17,3,3,3


In [35]:
S4D_df = df[df['Status'] == 'Slated for deactivation'] 
S4D_df = S4D_df['Affiliate Name'].value_counts()
S4D_df = S4D_df.transpose()
mail_merge_df['# of Slated for Deactivation Sites'] = S4D_df
mail_merge_df['# of Slated for Deactivation Sites'] = mail_merge_df['# of Slated for Deactivation Sites'].fillna(0)
mail_merge_df['# of Slated for Deactivation Sites'] = mail_merge_df['# of Slated for Deactivation Sites'].astype(int)

In [36]:
mail_merge_df.head()

,Number of Sites (High Level Active),Number of New Sites,Current Overall Training Percentage,Number of Providers Trained,# of SQCs Completed,# of Applications in Progress,# of Approved - Needs Training Sites,# of Approved - Waitlisted Sites,# of Covid Freeze Sites,# of Slated for Deactivation Sites
Affiliate Name,,,,,,,,,,
Independent / International,656,1,68%,2333,0,95,56,9,4,81
Reach Out and Read North Carolina,375,0,93%,2001,0,40,13,3,2,0
Reach Out and Read Colorado,329,1,92%,1635,0,13,4,10,0,0
Reach Out and Read Minnesota,301,1,86%,2385,0,17,6,0,1,0
Reach Out and Read Massachusetts,268,2,98%,1823,5,17,3,3,3,0


In [37]:
contact_list1 = []
contact_list2 = []
contact_list3 = []
affiliate_list1 = []
affiliate_list2 = []
affiliate_list3 = []

for ind, row in rslt_df.iterrows():
  email_check1 = rslt_df['Primary Contact Email'][ind]
  email_check2 = rslt_df['Medical Consultant Email'][ind]

  # Check if there is NO  Primary Contact Email
  if type(email_check1) == float:
    contact_list1.append(ind)
    affiliate_list1.append(rslt_df.loc[ind, "Affiliate Name"])
  
  # Check if there is NO Medical Consultant Email
  if type(email_check2) == float:
    contact_list2.append(ind)
    affiliate_list2.append(rslt_df.loc[ind, "Affiliate Name"])

  # Check if there is NO Primary Contcat Email AND NO medical contact email
  if type(email_check1) == float and type(email_check2) == float:
    contact_list3.append(ind)
    affiliate_list3.append(rslt_df.loc[ind, "Affiliate Name"])

contact_df1 = pd.DataFrame({'Missing Primary Contact': pd.Series(contact_list1), 'Affiliate Name': pd.Series(affiliate_list1)})
contact_df2 = pd.DataFrame({'Missing Medical Contact': pd.Series(contact_list2), 'Affiliate Name': pd.Series(affiliate_list2)})
contact_df3 = pd.DataFrame({'Missing Primary Contact and Medical Contact': pd.Series(contact_list3), 'Affiliate Name': pd.Series(affiliate_list3)})


In [38]:
contact_df1.head()

,Missing Primary Contact,Affiliate Name
0,40.3,Reach Out and Read Arizona
1,70.2,Reach Out and Read Bay Area
2,92,Reach Out and Read San Diego (California)
3,287.3,Reach Out and Read Illinois
4,287.4,Reach Out and Read Illinois


In [39]:
missing_primary_contact_df = contact_df1['Affiliate Name'].value_counts()
missing_primary_contact_df = missing_primary_contact_df.transpose()
mail_merge_df['# of Missing Primary Contacts'] = missing_primary_contact_df
mail_merge_df['# of Missing Primary Contacts'] = mail_merge_df['# of Missing Primary Contacts'].fillna(0)
mail_merge_df['# of Missing Primary Contacts'] = mail_merge_df['# of Missing Primary Contacts'].astype(int)

In [40]:
missing_medical_contact_df = contact_df2['Affiliate Name'].value_counts()
missing_medical_contact_df = missing_medical_contact_df.transpose()
mail_merge_df['# of Missing Medical Consultants'] = missing_medical_contact_df
mail_merge_df['# of Missing Medical Consultants'] = mail_merge_df['# of Missing Medical Consultants'].fillna(0)
mail_merge_df['# of Missing Medical Consultants'] = mail_merge_df['# of Missing Medical Consultants'].astype(int)

In [41]:
missing_primary_contact_df.head()

Reach Out and Read Illinois    29
Independent / International    25
Reach Out and Read Georgia     23
Reach Out and Read Iowa        19
Reach Out and Read Indiana     19
Name: Affiliate Name, dtype: int64

In [42]:
mail_merge_df.head()

,Number of Sites (High Level Active),Number of New Sites,Current Overall Training Percentage,Number of Providers Trained,# of SQCs Completed,# of Applications in Progress,# of Approved - Needs Training Sites,# of Approved - Waitlisted Sites,# of Covid Freeze Sites,# of Slated for Deactivation Sites,# of Missing Primary Contacts,# of Missing Medical Consultants
Affiliate Name,,,,,,,,,,,,
Independent / International,656,1,68%,2333,0,95,56,9,4,81,25,284
Reach Out and Read North Carolina,375,0,93%,2001,0,40,13,3,2,0,10,41
Reach Out and Read Colorado,329,1,92%,1635,0,13,4,10,0,0,13,103
Reach Out and Read Minnesota,301,1,86%,2385,0,17,6,0,1,0,11,47
Reach Out and Read Massachusetts,268,2,98%,1823,5,17,3,3,3,0,6,22


In [43]:
mail_merge_df = mail_merge_df.sort_values(by='Affiliate Name')

In [44]:
mail_merge_df = mail_merge_df.join(affiliate_contact_df.set_index('Affiliate Name'), on='Affiliate Name')

In [45]:
mail_merge_df.rename(index={'Reach Out and Read Florida (excluding Palm Beach County)':'Reach Out and Read Florida'},inplace=True)
mail_merge_df.rename(index={'Reach Out and Read Inland Empire (California)':'Reach Out and Read Inland Empire'},inplace=True)
mail_merge_df.rename(index={'Reach Out and Read Orange County (California)':'Reach Out and Read Orange County'},inplace=True)
mail_merge_df.rename(index={'Reach Out and Read Palm Beach County (Florida)':'Reach Out and Read Palm Beach County'},inplace=True)

In [46]:
# SAVE RESULTS DATAFRAME AS EXCEL FILE 
today = str(date.today())
excel_filename = "Mail_Merge_" + today + ".xlsx"
mail_merge_df.to_excel(excel_filename)
files.download(excel_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>